In [ ]:
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html


In [ ]:
!pip install accelerate


In [ ]:
!pip install transformers

In [ ]:
!pip install diffusers

In [ ]:
!pip install gradio

In [ ]:
from transformers.tools.base import Tool, get_default_device
from transformers.utils import is_accelerate_available
import torch

from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler

In [ ]:
TEXT_TO_IMAGE_DESCRIPTION = (
    "This is a tool that creates an image according to a prompt, which is a text description. It takes an input named `prompt` which "
    "contains the image description and outputs an image."
)

In [ ]:
class TextToImageTool(Tool):
    default_checkpoint = "runwayml/stable-diffusion-v1-5"
    description = TEXT_TO_IMAGE_DESCRIPTION
    inputs = ['text']
    outputs = ['image']

    def __init__(self, device=None, **hub_kwargs) -> None:
        if not is_accelerate_available():
            raise ImportError("Accelerate should be installed in order to use tools.")

        super().__init__()

        self.device = device
        self.pipeline = None
        self.hub_kwargs = hub_kwargs

    def setup(self):
        if self.device is None:
            self.device = get_default_device()

        self.pipeline = DiffusionPipeline.from_pretrained(self.default_checkpoint)
        self.pipeline.scheduler = DPMSolverMultistepScheduler.from_config(self.pipeline.scheduler.config)
        self.pipeline.to(self.device)

        if self.device.type == "cuda":
            self.pipeline.to(torch_dtype=torch.float16)

        self.is_initialized = True

    def __call__(self, prompt):
        if not self.is_initialized:
            self.setup()

        negative_prompt = "low quality, bad quality, deformed, low resolution"
        added_prompt = " , highest quality, highly realistic, very high resolution"

        return self.pipeline(prompt + added_prompt, negative_prompt=negative_prompt, num_inference_steps=25).images[0]




In [ ]:

import gradio as gr

tool = TextToImageTool()

def fn(*args, **kwargs):
    return tool(*args, **kwargs)

gr.Interface(
    fn=fn,
    inputs=tool.inputs,
    outputs=tool.outputs,
    title="TextToImageTool",
    article=tool.description,
).queue(concurrency_count=5).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fa3d435ce45973edbd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
